In [7]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Download daily gold futures data for the last 3 months
data = yf.download("GC=F", period="3mo", interval="1d")
close_prices = data['Close']['GC=F']
close_prices.head()

In [ ]:
returns = np.log(close_prices / close_prices.shift(1)).dropna()
mu = returns.mean()
sigma = returns.std()

pd.DataFrame({
    'Average daily return': [mu],
    'Daily volatility': [sigma]
})

In [ ]:
S0 = close_prices.iloc[-1]  # last closing price
T = 5                        # simulate 5 days
steps_per_day = 24            # hourly steps
steps = T * steps_per_day
paths = 500                  # number of simulation paths

dt = 1 / steps_per_day
sigma_hourly = sigma / np.sqrt(steps_per_day)
mu_hourly = mu / steps_per_day

simulations = np.zeros((steps, paths))
simulations[0] = S0

In [ ]:
for t in range(1, steps):
    Z = np.random.normal(size=paths)
    simulations[t] = simulations[t-1] * np.exp((mu_hourly - 0.5*sigma_hourly**2)*dt + sigma_hourly*np.sqrt(dt)*Z)

simulations.shape

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(simulations, color='lightblue', alpha=0.2)  # all paths faint
plt.plot(simulations.mean(axis=1), color='red', label='Average path')  # mean path
plt.title("Monte Carlo Simulation: GC=F (Gold Futures)")
plt.xlabel("Hourly steps")
plt.ylabel("Price")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8,5))
plt.hist(simulations[-1], bins=50, density=True, color='orange', alpha=0.7)
plt.title("Probability Distribution of Gold Price at Day 5")
plt.xlabel("Price")
plt.ylabel("Probability Density")
plt.show()

In [ ]:
pd.Series(simulations[-1]).describe()